# Language modelling

([Открыть в Colab](https://colab.research.google.com/drive/1c3QoUsRRCes9RKttQ3-SuSlFQZ3jAngz))


Обучим две различные символьные модели для генерации динозавров:
* модель на символьных биграмах
* ***RNN***-модель.


## Bigram model


In [1]:
!cat dinos.txt | tail

Zhuchengtyrannus
Ziapelta
Zigongosaurus
Zizhongosaurus
Zuniceratops
Zunityrannus
Zuolong
Zuoyunlong
Zupaysaurus
Zuul

In [2]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
print(names[:10])

['<aachenosaurus>', '<aardonyx>', '<abdallahsaurus>', '<abelisaurus>', '<abrictosaurus>', '<abrosaurus>', '<abydosaurus>', '<acanthopholis>', '<achelousaurus>', '<acheroraptor>']


In [3]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров

In [4]:
chars = [char for name in names for char in name]
freq = nltk.FreqDist(chars)

In [5]:
freq

FreqDist({'a': 2487, 's': 2285, 'u': 2123, 'o': 1710, 'r': 1704, '<': 1536, '>': 1536, 'n': 1081, 'i': 944, 'e': 913, ...})

In [6]:
print(list(freq.keys()))

['<', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', '>', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [7]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Define a function to estimate probabilty of character

In [8]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [9]:
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1160


Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [10]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

In [11]:
cfreq['a']

FreqDist({'u': 791, 'n': 347, 't': 204, 's': 171, 'l': 138, '>': 138, 'r': 124, 'c': 100, 'p': 89, 'm': 68, ...})

Оценим условные вероятности с помощью MLE.

In [12]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [13]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [55]:
cprob['a'].generate()

'n'

### Задание 1.

1) Напишите функцию, которая генерирует имя динозавра **фиксированной** длины. Используйте '<' как начальный символ.

2) Напишите функцию, которая генерирует имя динозавра любой дины.

In [56]:
import numpy as np

In [64]:
def fixed_name_generator(name_len):
    dinname = '<'
    for i in range(name_len):
        new_char = cprob[dinname[-1]].generate()

        while new_char in ['<', '>']:
            new_char = cprob[dinname[-1]].generate()

        dinname += new_char

    dinname += '>'
    return dinname

fixed_name_generator(8)

'<seeetoyl>'

In [100]:
def any_name_generator():
    dinname = '<'
    while dinname[-1] != '>':
        new_char = cprob[dinname[-1]].generate()

        while new_char in ['<']:
            new_char = cprob[dinname[-1]].generate()

        dinname += new_char

    return dinname

any_name_generator()

'<vopaursqurngurotalitodrs>'

In [136]:
def random_name_generator(max_len):
    dinname = '<'
    name_len = np.random.choice(max_len) + 1
    for i in range(name_len):
        new_char = cprob[dinname[-1]].generate()

        while new_char in ['<', '>']:
            new_char = cprob[dinname[-1]].generate()

        dinname += new_char

    dinname += '>'
    return dinname

random_name_generator(50)

'<tozaurngosarurapszionochelovesusausausauras>'

## Реккурентные нейронные сети (RNN)

Исходная последовательность:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

Для каждого входного значения $x_{1:i}$ получаем на выходе $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

Для всей последовательности $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ - рекурсивная функция активации, зависящая от двух параметров: $x_i$ и $s_{i-1}$ (вектор предыдущего состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- конкатенация $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

Построим языковую модель на основе RNN с помощью pytorch

In [137]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [138]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные

In [139]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [140]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [143]:
trn_ds.lines[1]

'aardonyx'

In [144]:
trn_ds.__getitem__(1)

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1

In [158]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [146]:
trn_ds.vocab_size

29

In [159]:
x, y = trn_ds[1]

In [160]:
x.shape

torch.Size([9, 29])

In [161]:
y.shape

torch.Size([9])

Опишем модель, функцию потерь и алгоритм оптимизации

In [150]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(h)
        return h, y

In [151]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

![rnn](images/dinos3.png)

In [152]:
def sample(model):
    model.eval()
    word_size=0
    end_char_idx = trn_ds.ch_to_idx['>'] # чтобы понимать, когда слово кончилось
    start_char_idx = trn_ds.ch_to_idx['<'] # seed для начала слова
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        
        # "сгенерирвали" начало слова
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != end_char_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float() # нетривиальное решение: подаём не всегда то же, что выдали
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [153]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

Обучим получившуюся модель

In [154]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [155]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [156]:
train(model, loss_fn, optimizer, epochs = 10)

Epoch:1
<
ag>
<hgiurohsapr>
<bnsrs>
<inrud>
<qel>
<<eotsaur>
<paros>
<taurug>
<amvntbshurus>
<lyurus>
<anras>
<auauadstaraurun>
<artotaurus>
<rupysaurud>
<lalrasaunus>

Epoch:2
<ahislhc>
<aaurudtseurus>
<zurortaraurus>
<htcltsaurus>
<ancrussurus>
<dtrkvpurus>
<hrcisauruc>
<shbresaurus>
<xstasurngssusus>
<suaopaurus>
<tcdssaurus>
<arcgyrueseurus>
<lyriosiprus>
<sapesaurun>
<ctamasar>

Epoch:3
<otlrus>
<drpnysaurun>
<manrasaurus>
<argulaesas>
<puattoratopy>
<lottanamrus>
<frcatocaurus>
<tcgvntdaurus>
<pnyooaurus>
<acucrnonrus>
<lltlgasaurus>
<stugisaurus>
<suanatrus>
<euadrsaurus>
<asaotaurus>

Epoch:4
<llosaurus>
<tcglaaurus>
<aaurasaurus>
<anxqtgiurus>
<ppuusaurus>
<aloasgsaurul>
<dqleasaurus>
<supneositaurus>
<hnaesasaurus>
<ararpicter>
<ouctsosaurus>
<mtrpasaqrus>
<huccspaurus>
<aterus>
<brstonaurus>

Epoch:5
<prramidaurus>
<aauraraurus>
<apwsrjiurus>
<rnwuntotah>
<amsasaurus>
<aiitenasaurus>
<strngssiysaurus>
<lcatasesapopc>
<cugicaorus>
<aussomaurus>
<qtuboraurus>
<tccsraurus>
<ata

In [186]:
print_sample(sample(model))

<fuaastariunhasaurus>


### Task 2.
Rewrite the sampling function so that pangrams (words that contain each character of the alphabet only once)

### Task 3.
Rewrite the sampling function so that is it is possible to change the sampling temperature

### Task 4.
Implement the beam search for sampling

# Reference

1. Sampling in  RNN: https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
2. Coursera course (main source): https://github.com/furkanu/deeplearning.ai-pytorch/tree/master/5-%20Sequence%20Models
3. Coursera course (main source): https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
4. LSTM: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
5. Туториал по модулю LSTM в pytorch: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html